# MobileNetV2 YOLOv3 for Text Detection

A text detection model based on MobileNetV2 and YoloV3.<br>
Pruned and quantized for deployment on edge devices.

**TODO:** Inference and "Show off" is way too little.<br>
**TODO:** Performance Evaluation

In [1]:
import os
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torchvision.models as models
import torch.nn.utils.prune as prune
from PIL import Image
from tqdm import tqdm
from torch.nn import functional as F
from torchvision import transforms
from torch.quantization import quantize_dynamic
from torch.utils.data import Dataset, DataLoader

---

## Model Definition

In [2]:
class MobileYOLOv3(nn.Module):
    def __init__(self, num_classes):
        super(MobileYOLOv3, self).__init__()
        self.num_classes = num_classes
        self.mobilenet = models.mobilenet_v2(weights='IMAGENET1K_V1').features
        self.conv1 = nn.Conv2d(1280, 512, kernel_size=1)
        self.conv2 = nn.Conv2d(512, 256, kernel_size=1)
        self.conv3 = nn.Conv2d(256, 5 * (num_classes + 5), kernel_size=1)  # 5 anchors, 5 for box coords + objectness

    def forward(self, x):
        x = self.mobilenet(x)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.conv3(x)
        return x.permute(0, 2, 3, 1).contiguous().view(x.size(0), 5, 8, 8, -1)  # (batch_size, 5, 8, 8, num_classes + 5)

---

## Loss Definition

In [3]:
class YoLoss(nn.Module):
    def __init__(self, num_classes):
        super(YoLoss, self).__init__()
        self.num_classes = num_classes
        self.mse_loss = nn.MSELoss(reduction='sum')
        self.bce_loss = nn.BCEWithLogitsLoss(reduction='sum')

    def forward(self, predictions, targets):
        obj_pred = predictions[..., 4:5]
        class_pred = predictions[..., 5:]
        bbox_pred = predictions[..., :4]

        obj_target = targets[..., 4:5]
        class_target = targets[..., 5:]
        bbox_target = targets[..., :4]

        obj_loss = self.bce_loss(obj_pred, obj_target)
        class_loss = self.bce_loss(class_pred, class_target)
        bbox_loss = self.mse_loss(bbox_pred, bbox_target)

        total_loss = obj_loss + class_loss + bbox_loss
        return total_loss / predictions.size(0)  # Normalize by batch size

---

## Training

In [4]:
class ICDAR2015(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])
        self.image_files = [f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        txt_name = 'gt_' + os.path.splitext(img_name)[0] + '.txt'
        with open(os.path.join(self.label_dir, txt_name), 'r') as f:
            lines = f.read().strip().split('\n')
        
        boxes = []
        for line in lines:
            parts = line.split(',')
            if len(parts) < 9:  # Skip invalid lines
                continue
            x1, y1, x2, y2, x3, y3, x4, y4 = map(float, parts[:8])
            x_min, y_min = min(x1, x2, x3, x4), min(y1, y2, y3, y4)
            x_max, y_max = max(x1, x2, x3, x4), max(y1, y2, y3, y4)
            boxes.append([x_min, y_min, x_max, y_max])
        
        boxes = torch.tensor(boxes, dtype=torch.float32)
        
        # Normalize box coordinates
        boxes[:, [0, 2]] /= image.shape[2]  # width
        boxes[:, [1, 3]] /= image.shape[1]  # height
        
        # Create target tensor (assuming 1 class for simplicity)class ICDAR2015(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])
        self.image_files = [f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        txt_name = 'gt_' + os.path.splitext(img_name)[0] + '.txt'
        txt_path = os.path.join(self.label_dir, txt_name)
        
        boxes = []
        try:
            with open(txt_path, 'r', encoding='utf-8-sig') as f:  # Use utf-8-sig to handle BOM
                lines = f.read().strip().split('\n')
            
            for line in lines:
                parts = line.split(',')
                if len(parts) < 8:  # Skip invalid lines
                    continue
                try:
                    x1, y1, x2, y2, x3, y3, x4, y4 = map(float, parts[:8])
                    x_min, y_min = min(x1, x2, x3, x4), min(y1, y2, y3, y4)
                    x_max, y_max = max(x1, x2, x3, x4), max(y1, y2, y3, y4)
                    boxes.append([x_min, y_min, x_max, y_max])
                except ValueError:
                    print(f"Skipping invalid box in {txt_name}: {parts[:8]}")
        except Exception as e:
            print(f"Error reading {txt_name}: {str(e)}")
            boxes = [[0, 0, 1, 1]]  # Default box if file can't be read
        
        boxes = torch.tensor(boxes, dtype=torch.float32)
        
        # Normalize box coordinates
        boxes[:, [0, 2]] /= image.shape[2]  # width
        boxes[:, [1, 3]] /= image.shape[1]  # height
        
        # Create target tensor (assuming 1 class for simplicity)
        target = torch.zeros((5, 8, 8, 6))  # 5 anchors, 8x8 grid, 6 values (4 for box, 1 for objectness, 1 for class)
        for box in boxes:
            i, j = min(int(box[1] * 8), 7), min(int(box[0] * 8), 7)  # Grid cell
            target[0, i, j, :4] = box
            target[0, i, j, 4] = 1  # objectness
            target[0, i, j, 5] = 1  # class (assuming 1 class)
        
        return image, target
        target = torch.zeros((5, 8, 8, 6))  # 5 anchors, 8x8 grid, 6 values (4 for box, 1 for objectness, 1 for class)
        for box in boxes:
            i, j = int(box[1] * 8), int(box[0] * 8)  # Grid cell
            target[0, i, j, :4] = box
            target[0, i, j, 4] = 1  # objectness
            target[0, i, j, 5] = 1  # class (assuming 1 class)
        
        return image, target

---

## Pruning & Quantization Definition

In [5]:
def prune_model(model, amount=0.3):
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            prune.l1_unstructured(module, name='weight', amount=amount)
            prune.remove(module, 'weight')
    return model

def quantize_model(model):
    return quantize_dynamic(model, {nn.Conv2d, nn.Linear}, dtype=torch.qint8)

---

## Training

In [6]:
batch_size = 32
num_workers = 4
num_classes = 1
learning_rate = 0.001
num_epochs = 100

# https://www.kaggle.com/datasets/bestofbests9/icdar2015
train_path = '/kaggle/input/icdar2015/ch4_training_images'
train_labels = '/kaggle/input/icdar2015/ch4_training_localization_transcription_gt'

train_dataset = ICDAR2015(train_path, train_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = MobileYOLOv3(num_classes=num_classes).to(device)
criterion = YoLoss(num_classes=num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):        
        data, targets = data.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

        if batch_idx % 10 == 0:
            print(f"Batch {batch_idx}, Loss: {loss.item()}")
        
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(train_loader):.4f}')

# Prune, then Quantize
quantized_model = quantize_model(prune_model(model))
    
# Save the model
torch.save(quantized_model.state_dict(), 'pq_yolov3_mobilenetv2.pth')

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


cuda


100%|██████████| 13.6M/13.6M [00:00<00:00, 19.3MB/s]
  6%|▋         | 2/32 [00:02<00:33,  1.13s/it]

Batch 0, Loss: 528.10009765625


 38%|███▊      | 12/32 [00:04<00:03,  5.72it/s]

...

Batch 10, Loss: 2.452746868133545


 69%|██████▉   | 22/32 [00:04<00:01,  5.19it/s]

Batch 20, Loss: 1.780097484588623


100%|██████████| 32/32 [00:06<00:00,  5.07it/s]


Batch 30, Loss: 2.234168529510498
Epoch [100/100], Loss: 2.3992


In [10]:
test_input = torch.randn(1, 3, 256, 256).to(device)
quantized_model.eval()
with torch.no_grad():
    output = quantized_model(test_input)
print(output.shape)

torch.Size([1, 5, 8, 8, 6])
